In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))
Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
df_feats_clinical_icd = pd.merge(df_feats_clinical,
                                 df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [7]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [8]:
df_feats_clinical_icd_train = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_feats_clinical_icd_val = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_feats_clinical_icd_test = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_feats_clinical_icd_train)+len(df_feats_clinical_icd_val)+len(df_feats_clinical_icd_test)-len(df_feats_clinical_icd)

0

In [9]:
X_train = df_feats_clinical_icd_train
X_val = df_feats_clinical_icd_val
X_train_val = pd.concat([X_train,X_val])
X_test = df_feats_clinical_icd_test

y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_train_val = pd.concat([y_train,y_val])
y_test = Outcome_labels_test

In [10]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4306),
 (13665, 4306),
 (122983, 4306),
 (13664, 4306),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [11]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [12]:
del sys.modules["utilities"]
from utilities import *

In [13]:
experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[666])

100%|████████████████████████████████████████| 1/1 [4:37:29<00:00, 16649.91s/it]


A. Scores on training set
avg XGBoost classifer: auc/  0.8867  F1:  0.6166  precision:  0.7613  recall:  0.5181  accuracy:  0.8417
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.842  F1:  0.5314  precision:  0.6949  recall:  0.4303  accuracy:  0.8136
std MLP classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg logistic classifer: auc/  0.8551  F1:  0.6279  precision:  0.5334  recall:  0.7631  accuracy:  0.7778
std logistic classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
****************************************************

B. Scores on test set
avg XGBoost classifer: auc/  0.8572  F1:  0.6052  precision:  0.7318  recall:  0.516  accuracy:  0.8057
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.8375  F1:  0.5747  precision:  0.7201  recall:  0.4782  accuracy:  0.7957
std MLP classifer: auc/  